In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import sys
sys.path.insert(0,"../input/wheatutils/yolov5-master-pure/yolov5-master")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from utils import google_utils
from utils.datasets import *
from utils.utils import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Configure**

In [ ]:

def run_wbf(boxes, scores, image_size=1023, iou_thr=0.5, skip_box_thr=0.7, weights=None):
    #boxes = [prediction[image_index]['boxes'].data.cpu().numpy()/(image_size-1) for prediction in predictions]
    #scores = [prediction[image_index]['scores'].data.cpu().numpy() for prediction in predictions]
    labels = [np.zeros(score.shape[0]) for score in scores]
    boxes = [box/(image_size) for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #boxes, scores, labels = nms(boxes, scores, labels, weights=[1,1,1,1,1], iou_thr=0.5)
    boxes = boxes*(image_size)
    return boxes, scores, labels

def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

def detect1Image(im0, imgsz, model, device, conf_thres, iou_thres):
    img = letterbox(im0, new_shape=imgsz)[0]
    # Convert
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img =  img.float()  # uint8 to fp16/32
    img /= 255.0   
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Inference
    pred = model(img, augment=False)[0]

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres)

    boxes = []
    scores = []
    for i, det in enumerate(pred):  # detections per image
        # save_path = 'draw/' + image_id + '.jpg'
        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            # Write results
            for *xyxy, conf, cls in det:
                boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                scores.append(conf)

    return np.array(boxes), np.array(scores) 

In [ ]:
target_hist = [[709,0,29,149,8,650,854,17,1532,224,319,2643,759,5639,3137,2741,6813,1621,2307,7205,2698,11966,4556,6124,4065,11436,9032,1096,2116,2476,21879,1433,6860,5239,14507,24854,9760,12524,19310,19484,24916,19586,25474,37620,28478,30689,40834,28251,42013,30700,35224,37839,30140,31922,24836,34327,28494,23063,21333,18109,20119,12411,17297,20126,10381,11759,16619,14940,6770,17773,7031,6054,7837,5466,10190,3770,4382,5194,679,2022,6769,1132,951,1318,2768,995,453,2119,517,279,1718,240,270,478,102,763,502,39,341,105,48,330,45,273,93,41,112,16,17,94,179,72,19,32,106,53,16,0,1,0,205,0,2,0,16,45,55,0,3,0,8,0,53,16,0,3,0,0,4,0,119,10,3,0,0,24,12,5,1,0,15,0,1,6,0,0,66,0,0,0,39,2,0,23,4,288,1,0,0,27,189,0,0,191,18,19,0,252,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[340,2,8,17,32,63,98,120,162,160,207,304,418,489,553,651,723,857,1064,1216,1483,1724,1705,1875,1943,1998,2270,2239,2569,2473,2560,2654,3004,3027,2920,2895,3318,2913,3487,3397,3680,3530,3341,3949,4079,4111,4384,4470,4521,4698,4557,5080,4598,5367,4755,5675,5032,5364,6146,5855,6037,6378,6207,5712,7754,6290,6481,7644,6859,6605,8910,8369,6993,9633,8270,8426,8122,10626,10783,7943,9159,11868,10393,10482,8023,17871,8235,10538,11650,12445,13544,10063,11366,12589,13245,9114,13864,12585,11667,11404,12560,9429,15865,9455,11715,10499,12104,12521,11367,11314,8007,10997,11345,12327,8994,9322,9441,10355,9430,8092,10228,9169,8947,9216,8434,6804,4399,9155,14123,4004,5893,7453,7418,6617,7280,6767,5364,6619,5525,5116,5846,4570,5303,5998,3825,4079,5221,4204,3991,4970,3150,4082,2456,4520,2269,4185,2842,2398,3039,3285,1545,2663,2982,2137,1700,2112,2148,2198,1455,653,3972,592,1029,1905,1646,1025,794,1557,672,1726,939,1125,1155,679,1011,1083,563,560,1204,681,378,1289,164,560,577,524,530,640,735,276,647,434,436,347,686,223,543,323,275,521,356,363,163,435,353,280,275,342,134,417,260,253,266,308,255,148,189,292,224,131,281,167,176,199,193,290,137,194,155,135,165,143,167,188,166,142,156,154,155,87,86,25,7,0,0,6765],
[0,0,3,3,11,10,16,29,22,60,114,190,404,700,1369,2295,3801,5877,7986,9680,11574,12894,13903,14184,14494,14412,13601,13549,12918,12379,12174,12048,11818,11625,11566,11261,11056,10916,10269,10301,9615,9290,8788,8550,8130,7821,7531,7273,7088,6850,6931,6875,6603,6667,6462,6456,6235,6287,6186,6223,5901,6081,6292,6163,6234,6039,6218,5826,5914,5803,5622,5615,5542,5439,5226,4979,4726,4766,4693,4592,4542,4500,4535,4700,4608,4582,4516,4493,4337,4290,4231,4141,3978,3893,3731,3606,3433,3472,3458,3544,3455,3519,3657,3610,3667,3863,3810,3915,3947,3896,3885,3782,3780,3553,3665,3694,3659,3764,3727,3746,3832,3907,3840,3980,4026,4051,4124,4307,4022,4019,4030,3827,3880,3984,3799,3946,3874,3900,3799,3880,3924,3677,3730,3661,3730,3599,3564,3529,3715,3626,3615,3627,3533,3404,3410,3413,3327,3193,3083,3075,3156,3164,3038,3094,3077,3126,3070,3029,3007,3131,3120,3029,3030,3126,2931,3007,2971,2883,2917,2786,2845,2784,2694,2752,2735,2731,2667,2774,2708,2716,2737,2647,2606,2637,2632,2705,2747,2644,2708,2759,2696,2732,2706,2732,2709,2678,2865,2700,2653,2558,2576,2560,2429,2492,2407,2394,2368,2311,2267,2113,2023,1961,1950,1791,1761,1713,1745,1597,1591,1528,1381,1385,1376,1314,1244,1136,1122,1071,943,895,851,800,774,744,735,666,613,533,499,450,416,370,351,316,277,1098]]
target_hist = [np.array(x) for x in target_hist]
def img_hist(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist_lst = []
    save = False
    if save:
        file = open('target_hist.txt','w')
    for i in range(3):
        hist, _ = np.histogram(img_hsv[:, :, i].ravel(), 256, [0, 256])
        if save:
            s = '['+','.join([str(i) for i in hist])+']\n'
            file.write(s)
        hist_lst.append(hist)
    if save:
        file.close()
    return hist_lst


def aug_hist(src_img):
    color = ('h', 's', 'v')
    hist1_lst = img_hist(src_img)
    hist2_lst = target_hist
    src_hsv = cv2.cvtColor(src_img, cv2.COLOR_BGR2HSV)  # bgr转hsv
    for i, col in enumerate(color):
        # hist1 = cv2.calcHist([src_hsv], [i], None, [256], [0, 256])
        hist1 = hist1_lst[i]
        hist2 = hist2_lst[i]
        cdf1 = hist1.cumsum()  # 灰度值0-255的累计值数组
        cdf2 = hist2.cumsum()
        cdf1_hist = hist1.cumsum() / cdf1.max()  # 灰度值的累计值的比率
        cdf2_hist = hist2.cumsum() / cdf2.max()
        diff_cdf = np.zeros([256, 256])  # diff_cdf 里是每2个灰度值比率间的差值
        for j in range(256):
            diff_cdf[j, :] = np.abs(cdf1_hist[j] - cdf2_hist)

        lut = np.zeros(256)  # 映射表
        for j in range(256):
            # 直方图规定化的映射原理
            index = int(np.argmin(diff_cdf[j,:]))
            lut[j] = index
        h = int(src_hsv.shape[0])
        w = int(src_hsv.shape[1])
        for j in range(h):  # 对原图像进行灰度值的映射
            src_hsv[j, :, i] = lut[src_hsv[j, :, i]]

    aug_img = cv2.cvtColor(src_hsv, cv2.COLOR_HSV2BGR)  # hsv转bgr
    return aug_img

In [ ]:


parser = {}
parser['weights']='../input/best-train/best.pt'
parser['source']='../input/global-wheat-detection/test'#/796707dd7.jpg
parser['output']='/kaggle/input/global-wheat/'
parser['img_size']=1024
parser['conf_thresh']=0.35
parser['iou_thresh']=0.6
parser['agnostic_nms']=False
parser['classes']=None
parser['view_img']=True
opt = parser
show = 0
aug_infer = 0
#print(opt)
def detect(save_img=False):
    out, source, weights, imgsz ,view_img= \
        opt['output'], opt['source'], opt['weights'], opt['img_size'],opt['view_img']


    # Initialize
    #device = opt['device']#torch_utils.select_device(opt['device'])
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load model
    model = torch.load(weights, map_location=device)['model'].float()  # load to FP32
    # torch.save(torch.load(weights, map_location=device), weights)  # update model if SourceChangeWarning
    # model.fuse()
    model.to(device).eval()

    # Second-stage classifier
    classify = False
    if classify:
        modelc = torch_utils.load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Set Dataloader

    dataset = LoadImages(source, img_size=imgsz)

    # Get names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img) if device.type != 'cpu' else None  # run once
    image_id_lst = [p.strip('.jpg') for p in os.listdir(source)]
    df = pd.DataFrame(columns=['PredictionString'], index=image_id_lst)
    fid = 0
    for path, img, im0s, vid_cap in dataset:
        if show:
            fid += 1
            plt.figure(fid)
        image_id = path[path.rindex('/')+1:len(path)].strip('.jpg')
        if aug_infer:
            img = np.transpose(aug_hist(np.transpose(img,[1,2,0])),[2,0,1])
        img = torch.from_numpy(img).to(device)
        img = img.float()  # uint8 to fp32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
            
        # Inference
        t1 = torch_utils.time_synchronized()
        pred = model(img, augment=False)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt['conf_thresh'], opt['iou_thresh'], classes=opt['classes'], agnostic=opt['agnostic_nms'])
        t2 = torch_utils.time_synchronized()

#         # Apply Classifier
#         if classify:
#             pred = apply_classifier(pred, modelc, img, im0s)

#         # Process detections
        result = []
        
        for i, det in enumerate(pred):  # detections per image
            if det is not None and len(det):
                if show:
                    plt.imshow(np.transpose(img.cpu().numpy()[0],[1,2,0]))

                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                for xmin,ymin,xmax,ymax, conf, cls in det:
                    score = round(float(conf),2)
                    width = xmax - xmin
                    height = ymax - ymin
                    result.append(' '.join(list(map(str, [score, int(xmin), int(ymin), int(width), int(height)]))))
                    if show:
                        plt.gca().add_patch(
                            plt.Rectangle((int(xmin), int(ymin)), int(xmax) - int(xmin),
                              int(ymax) - int(ymin), fill=False,
                              edgecolor='r', linewidth=3)
                        )
            df.loc[image_id, 'PredictionString'] = ' '.join(result)
        print('Done. (%.3fs)' % (t2 - t1))
    df.reset_index(inplace=True, drop=False)
    df.columns = ['image_id','PredictionString']
    df.to_csv('submission.csv', index=False)
    

with torch.no_grad():
    detect()
